<a href="https://colab.research.google.com/github/Shoukhin1803078/My-research-/blob/main/BDMLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -q peft
#!pip show peft

In [2]:
#import peft
#dir(peft)

In [3]:
!pip install -q bitsandbytes peft trl accelerate torchkeras langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [4]:
'''

#from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM



import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
import torch
from torch import nn
from torch.utils.data import DataLoader

from time import time
import warnings
warnings.filterwarnings('ignore')

# Import specific libraries for acceleration and model performance monitoring
import accelerate
import peft

# Import necessary modules from the transformers library
from transformers import AutoTokenizer, AutoConfig, AutoModel, BitsAndBytesConfig, AutoModelForCausalLM
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

# Set environment variables for PyTorch CUDA and XLA
# max_split_size_mb - prevents allocator from splitting blocks larger than this size
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"
os.environ['XLA_USE_BF16'] = "1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

# Import the torchkeras library for enhanced PyTorch functionality
import torchkeras

time_start = time()
'''

'\n\n#from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM\n\n\n\nimport numpy as np\nimport pandas as pd \nfrom datasets import load_dataset, Dataset\nimport torch\nfrom torch import nn \nfrom torch.utils.data import DataLoader \n\nfrom time import time\nimport warnings \nwarnings.filterwarnings(\'ignore\')\n\n# Import specific libraries for acceleration and model performance monitoring\nimport accelerate \nimport peft \n\n# Import necessary modules from the transformers library\nfrom transformers import AutoTokenizer, AutoConfig, AutoModel, BitsAndBytesConfig, AutoModelForCausalLM\nfrom transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy\n\n# Set environment variables for PyTorch CUDA and XLA\n# max_split_size_mb - prevents allocator from splitting blocks larger than this size\nimport os\nos.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64" \nos.environ[\'XLA_USE_BF16\'] = "1"\nos.

In [5]:
!pip show datasets

Name: datasets
Version: 2.18.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyarrow-hotfix, pyyaml, requests, tqdm, xxhash
Required-by: trl


In [6]:
!pip install --upgrade datasets

In [7]:
!pip install datasets

In [8]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset, Dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)
import bitsandbytes as bnb


import pandas as pd
import numpy as np
from time import time

import warnings
warnings.filterwarnings('ignore')
from accelerate import Accelerator

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer

Dataset preprocessing


In [14]:
import json
# Read the first few lines of the file to check the format
with open("/content/sample_data/News_Category_Dataset_v3.json", 'r') as file:
    for _ in range(5):
        print(file.readline())

# Initialize an empty list to store the JSON objects
data = []



{"link": "https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9", "headline": "Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters", "category": "U.S. NEWS", "short_description": "Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.", "authors": "Carla K. Johnson, AP", "date": "2022-09-23"}

{"link": "https://www.huffpost.com/entry/american-airlines-passenger-banned-flight-attendant-punch-justice-department_n_632e25d3e4b0e247890329fe", "headline": "American Airlines Flyer Charged, Banned For Life After Punching Flight Attendant On Video", "category": "U.S. NEWS", "short_description": "He was subdued by passengers and crew when he fled to the back of the aircraft after the confrontation, according to the U.S. attorney's office in Los Angeles.", "authors": "Mary Papenfuss", "date": "2022-09-23"}

{"link": "https://www.huffpost.com/ent

In [16]:
import json

data = []

with open('/content/sample_data/News_Category_Dataset_v3.json', 'r') as file:
    for line in file:
        try:
            # Attempt to parse the JSON line
            parsed_line = json.loads(line)
            data.append(parsed_line)
        except json.JSONDecodeError as e:
            # Print an error message and continue with the next line
            print(f"Error parsing line: {e}")
            continue

raw_news_df = pd.DataFrame(data)
print(raw_news_df.head())


Error parsing line: Expecting property name enclosed in double quotes: line 1 column 343 (char 342)
                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  H

In [17]:
print(raw_news_df['category'].unique().size)
print(raw_news_df['category'].unique())

42
['U.S. NEWS' 'COMEDY' 'PARENTING' 'WORLD NEWS' 'CULTURE & ARTS' 'TECH'
 'SPORTS' 'ENTERTAINMENT' 'POLITICS' 'WEIRD NEWS' 'ENVIRONMENT'
 'EDUCATION' 'CRIME' 'SCIENCE' 'WELLNESS' 'BUSINESS' 'STYLE & BEAUTY'
 'FOOD & DRINK' 'MEDIA' 'QUEER VOICES' 'HOME & LIVING' 'WOMEN'
 'BLACK VOICES' 'TRAVEL' 'MONEY' 'RELIGION' 'LATINO VOICES' 'IMPACT'
 'WEDDINGS' 'COLLEGE' 'PARENTS' 'ARTS & CULTURE' 'STYLE' 'GREEN' 'TASTE'
 'HEALTHY LIVING' 'THE WORLDPOST' 'GOOD NEWS' 'WORLDPOST' 'FIFTY' 'ARTS'
 'DIVORCE']


In [18]:
raw_news_df['headline'][0]

'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters'

In [20]:
instr_news_df = pd.DataFrame()

In [21]:
instr_news_df['input'] = raw_news_df['headline'].astype(str) + '\n' + raw_news_df['short_description'].astype(str)
instr_news_df['output'] = raw_news_df['category']
instr_news_df['instruction'] = 'Categorize the news article description into one of the 42 categories:\n\nU.S. NEWS\nCOMEDY\nPARENTING\nWORLD NEWS\nCULTURE & ARTS\nTECH\nSPORTS\nENTERTAINMENT\nPOLITICS\nWEIRD NEWS\nENVIRONMENT\nEDUCATION\nCRIME\nSCIENCE\nWELLNESS\nBUSINESS\nSTYLE & BEAUTY\nFOOD & DRINK\nMEDIA\nQUEER VOICES\nHOME & LIVING\nWOMEN\nBLACK VOICES\nTRAVEL\nMONEY\nRELIGION\nLATINO VOICES\nIMPACT\nWEDDINGS\nCOLLEGE\nPARENTS\nARTS & CULTURE\nSTYLE\nGREEN\nTASTE\nHEALTHY LIVING\nTHE WORLDPOST\nGOOD NEWS\nWORLDPOST\nFIFTY\nARTS\nDIVORCE'


In [22]:
instr_news_df.head()

,input,output,instruction
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Categorize the news article description into o...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,Categorize the news article description into o...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,Categorize the news article description into o...
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,Categorize the news article description into o...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Categorize the news article description into o...


In [23]:
instr_news_df.to_csv("news_classification.csv", index=False)

In [24]:
news_df = pd.read_csv('news_classification.csv')

In [25]:
news_df.head()

,input,output,instruction
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Categorize the news article description into o...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,Categorize the news article description into o...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,Categorize the news article description into o...
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,Categorize the news article description into o...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Categorize the news article description into o...


In [26]:
news_df['input'][0]

'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters\nHealth experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.'

**Model Setup**



In [27]:
#model_name = "/kaggle/input/llama-2/pytorch/7b-hf/1"
model_name = "tiiuae/falcon-7b-instruct"

In [28]:
'''

time_1 = time()

# Set model configuration for Kaggle compatibility
model_name_or_path = model_name
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Loads model in 4 bit quantization
    bnb_4bit_use_double_quant=True,         # Uses second quantization to save 0.4 bits
    bnb_4bit_quant_type="nf4",              # 4-bit NoramlFloat
    bnb_4bit_compute_dtype=torch.bfloat16,  # Computation still happens in 16 or 32-bit
    llm_int8_has_fp16_weight=False,

)

# Initialize tokenizer with the specified model configuration
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=True)  # Cache to the current working directory
tokenizer.pad_token = tokenizer.eos_token

# Initialize the model with the specified model configuration and device mapping
model =  AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map = 'auto')
model.config.use_cache = False     # Disable cache for generation
model.config.pretraining_tp = 1    # Slower but more accurate computation of linear layers

time_2 = time()
print(f"Time elapsed: {round(time_2-time_1, 3)} sec.")
'''

'\n\ntime_1 = time()\n\n# Set model configuration for Kaggle compatibility\nmodel_name_or_path = model_name\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,                      # Loads model in 4 bit quantization\n    bnb_4bit_use_double_quant=True,         # Uses second quantization to save 0.4 bits\n    bnb_4bit_quant_type="nf4",              # 4-bit NoramlFloat\n    bnb_4bit_compute_dtype=torch.bfloat16,  # Computation still happens in 16 or 32-bit\n    llm_int8_has_fp16_weight=False,\n        \n)\n\n# Initialize tokenizer with the specified model configuration\ntokenizer = AutoTokenizer.from_pretrained(\n    model_name_or_path, trust_remote_code=True)  # Cache to the current working directory\ntokenizer.pad_token = tokenizer.eos_token\n\n# Initialize the model with the specified model configuration and device mapping\nmodel =  AutoModelForCausalLM.from_pretrained(\n    model_name_or_path,\n    quantization_config=bnb_config,\n    trust_remote_code=True,\n    low_cpu_mem_u

**Or function defined load model**

In [29]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

def load_model(model_name, bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """


    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'


    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

# transformers parameters

model_name = model_name



load_in_4bit = True


bnb_4bit_use_double_quant = True

bnb_4bit_quant_type = "nf4"

bnb_4bit_compute_dtype = torch.bfloat16



bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

model, tokenizer = load_model(model_name, bnb_config)

In [30]:
#dataset = load_dataset("csv", data_files='news_classification.csv')
from datasets import Dataset
news_df = pd.read_csv('news_classification.csv')
dataset = Dataset.from_pandas(news_df)

print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Number of prompts: 134481
Column names are: ['input', 'output', 'instruction']


In [31]:
def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """


    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)

    sample["text"] = formatted_prompt

    return sample

In [32]:
print(create_prompt_formats(dataset[0]))

{'input': 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters\nHealth experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.', 'output': 'U.S. NEWS', 'instruction': 'Categorize the news article description into one of the 42 categories:\n\nU.S. NEWS\nCOMEDY\nPARENTING\nWORLD NEWS\nCULTURE & ARTS\nTECH\nSPORTS\nENTERTAINMENT\nPOLITICS\nWEIRD NEWS\nENVIRONMENT\nEDUCATION\nCRIME\nSCIENCE\nWELLNESS\nBUSINESS\nSTYLE & BEAUTY\nFOOD & DRINK\nMEDIA\nQUEER VOICES\nHOME & LIVING\nWOMEN\nBLACK VOICES\nTRAVEL\nMONEY\nRELIGION\nLATINO VOICES\nIMPACT\nWEDDINGS\nCOLLEGE\nPARENTS\nARTS & CULTURE\nSTYLE\nGREEN\nTASTE\nHEALTHY LIVING\nTHE WORLDPOST\nGOOD NEWS\nWORLDPOST\nFIFTY\nARTS\nDIVORCE', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the news article description into one of the 42 c

In [33]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [35]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

In [36]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """


    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["instruction", "input", "output", "text"],
    )


    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)


    dataset = dataset.shuffle(seed = seed)

    return dataset

**Create peft config**

In [38]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [39]:
accelerator = Accelerator()
def fine_tune(model,
          tokenizer,
          dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """

    model.gradient_checkpointing_enable()


    model = prepare_model_for_kbit_training(model)

    # Get LoRA module names
    target_modules = find_all_linear_names(model)

    # Create PEFT configuration for these modules and wrap the model to PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = accelerator.prepare(Trainer(
        model = model,
        train_dataset = dataset,
        args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
            report_to = "none"
        ),
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    ))

    model.config.use_cache = False

    do_train = True


    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)


    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

In [40]:

# QLoRA parameters
# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.1


bias = "none"

task_type = "CAUSAL_LM"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Batch size per GPU for training
per_device_train_batch_size = 1

gradient_accumulation_steps = 4

learning_rate = 2e-4

optim = "paged_adamw_32bit"
max_steps = 20
warmup_steps = 2
fp16 = True
logging_steps = 1

In [41]:
fine_tune(model, tokenizer, preprocessed_dataset, lora_r, lora_alpha, lora_dropout, bias, task_type, per_device_train_batch_size, gradient_accumulation_steps, warmup_steps, max_steps, learning_rate, fp16, logging_steps, output_dir, optim)

In [42]:

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map = "auto", torch_dtype = torch.bfloat16)

model = model.merge_and_unload()


output_merged_dir = "results/news_classification_llama2_7b/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok = True)
model.save_pretrained(output_merged_dir, safe_serialization = True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)

In [ ]:
time_1 = time()


model_name_or_path = '/kaggle/working/results/news_classification_llama2_7b/final_merged_checkpoint'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_has_fp16_weight=False,

)

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


model =  AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map = 'auto')
model.config.use_cache = False
model.config.pretraining_tp = 1

time_2 = time()
print(f"Time elapsed: {round(time_2-time_1, 3)} sec.")

In [ ]:
time_1 = time()


input_text = 'Who is Charlie Brown from Peanuts?'


inputs = tokenizer(input_text)
inputs = {k: torch.tensor([v]) for k, v in inputs.items()}


outputs = model.generate(**inputs, max_new_tokens=128, repetition_penalty=1.1)

generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text)

time_2 = time()
print(f"Time elapsed: {round(time_2-time_1, 3)} sec.")

In [ ]:
time_1 = time()


input_text = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the news article description into one of the 42 categories:\n\nU.S. NEWS\nCOMEDY\nPARENTING\nWORLD NEWS\nCULTURE & ARTS\nTECH\nSPORTS\nENTERTAINMENT\nPOLITICS\nWEIRD NEWS\nENVIRONMENT\nEDUCATION\nCRIME\nSCIENCE\nWELLNESS\nBUSINESS\nSTYLE & BEAUTY\nFOOD & DRINK\nMEDIA\nQUEER VOICES\nHOME & LIVING\nWOMEN\nBLACK VOICES\nTRAVEL\nMONEY\nRELIGION\nLATINO VOICES\nIMPACT\nWEDDINGS\nCOLLEGE\nPARENTS\nARTS & CULTURE\nSTYLE\nGREEN\nTASTE\nHEALTHY LIVING\nTHE WORLDPOST\nGOOD NEWS\nWORLDPOST\nFIFTY\nARTS\nDIVORCE\n\nInput:\nOver 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters\nHealth experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.\n\n### Response:'


inputs = tokenizer(input_text)
inputs = {k: torch.tensor([v]) for k, v in inputs.items()}

outputs = model.generate(**inputs, max_new_tokens=128, repetition_penalty=1.1)

generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text)

time_2 = time()
print(f"Time elapsed: {round(time_2-time_1, 3)} sec.")

In [43]:
create_prompt_formats(dataset[0])['text']

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the news article description into one of the 42 categories:\n\nU.S. NEWS\nCOMEDY\nPARENTING\nWORLD NEWS\nCULTURE & ARTS\nTECH\nSPORTS\nENTERTAINMENT\nPOLITICS\nWEIRD NEWS\nENVIRONMENT\nEDUCATION\nCRIME\nSCIENCE\nWELLNESS\nBUSINESS\nSTYLE & BEAUTY\nFOOD & DRINK\nMEDIA\nQUEER VOICES\nHOME & LIVING\nWOMEN\nBLACK VOICES\nTRAVEL\nMONEY\nRELIGION\nLATINO VOICES\nIMPACT\nWEDDINGS\nCOLLEGE\nPARENTS\nARTS & CULTURE\nSTYLE\nGREEN\nTASTE\nHEALTHY LIVING\nTHE WORLDPOST\nGOOD NEWS\nWORLDPOST\nFIFTY\nARTS\nDIVORCE\n\nInput:\nOver 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters\nHealth experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.\n\n### Response:\nU.S. NEWS\n\n### End'

In [45]:
time_end = time()